# Multi-Rate Sensor Fusion

This notebook demonstrates multi-rate Extended Kalman Filter (EKF) sensor fusion for the Benji rover. The key insight is that different sensors operate at different rates:

| Sensor | Rate | Measurements |
|--------|------|-------------|
| IMU (gyro) | 50 Hz | $\theta$, $\dot{\theta}$ |
| Wheel encoders | 50 Hz | $v_l$, $v_r$ |
| Accelerometer | 50 Hz | $a_x$, $a_y$ |
| Camera | 5 Hz | $x$, $y$ |

The EKF handles this naturally: predict at the control rate, update with whatever measurements are available.

We compare trajectory tracking with and without camera position updates across all three controllers (LQR+FF, QP-MPC, NL-MPC).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_discrete_are
import time

from utils import angle_wrap, state_error, generate_smooth_reference_trajectory
from kalman import MultiRateEKF
from mpc_controller import QPMPC, NonlinearMPC, compute_jacobians_at_state

## Parameters

In [ ]:
# Robot parameters
L = 0.14        # wheelbase [m]
tau = 0.2       # motor time constant [s]
v_cruise = 0.25 # cruise speed [m/s]
u_max = 0.5     # max wheel command [m/s]

# State dimensions
n_x, n_u = 5, 2

# Simulation parameters
T_final = 60.0
dt = 0.02       # 50 Hz control/estimation rate
t = np.arange(0, T_final, dt)
N = len(t)

# Camera parameters
camera_rate = 5.0  # Hz
camera_interval = int((1.0 / camera_rate) / dt)  # steps between camera updates

# Process noise covariance
Q_process = np.diag([1e-5, 1e-5, 1e-6, 1e-4, 1e-4])
Q_chol = np.linalg.cholesky(Q_process)

# IMU + Encoder measurement noise (always available)
# Measurements: [theta, theta_dot, v_l, v_r, a_x, a_y]
R_imu = np.diag([1e-4, 1e-3, 1e-4, 1e-4, 1e-2, 1e-2])

# Camera measurement noise
sigma_camera = 0.05  # 5 cm position noise
R_camera = np.diag([sigma_camera**2, sigma_camera**2])

print(f"Control rate: {1/dt:.0f} Hz")
print(f"Camera rate: {camera_rate:.0f} Hz (every {camera_interval} steps)")
print(f"Camera noise: {sigma_camera*100:.0f} cm (1-sigma)")

In [ ]:
def nonlinear_dynamics(x, u, dt_step, L, tau):
    """
    Propagate state using nonlinear tank drive dynamics.
    
    State: [x, y, theta, v_l, v_r]
    Input: [u_l, u_r] (wheel velocity commands)
    """
    x_pos, y_pos, theta, v_l, v_r = x
    u_l, u_r = u
    
    v_avg = (v_l + v_r) / 2.0
    
    x_dot = v_avg * np.cos(theta)
    y_dot = v_avg * np.sin(theta)
    theta_dot = (v_r - v_l) / L
    v_l_dot = (-v_l + u_l) / tau
    v_r_dot = (-v_r + u_r) / tau
    
    return np.array([
        x_pos + x_dot * dt_step,
        y_pos + y_dot * dt_step,
        theta + theta_dot * dt_step,
        v_l + v_l_dot * dt_step,
        v_r + v_r_dot * dt_step
    ])

## Extended Kalman Filter

The EKF uses:
- **Nonlinear dynamics** for prediction
- **Linearized measurement models** for updates

The IMU+encoder measurement includes the accelerometer, which provides body-frame accelerations:

$$a_x = \frac{\dot{v}_l + \dot{v}_r}{2} = \frac{1}{2\tau}\left((u_l - v_l) + (u_r - v_r)\right)$$
$$a_y = \frac{v_l + v_r}{2} \cdot \dot{\theta} = \frac{v_l + v_r}{2} \cdot \frac{v_r - v_l}{L}$$

The `MultiRateEKF` class is imported from `kalman.py`. See the module for implementation details.

## Controllers

Set up all three controllers for comparison.

In [ ]:
# LQR setup
Q_lqr = np.diag([20.0, 20.0, 5.0, 0.1, 0.1])
R_lqr = np.diag([1.0, 1.0])

x_nom = np.array([0, 0, 0, v_cruise, v_cruise])
A_c, B_c = compute_jacobians_at_state(x_nom, L, tau)
A_d = np.eye(n_x) + dt * A_c
B_d = dt * B_c

P_dare = solve_discrete_are(A_d, B_d, Q_lqr, R_lqr)
K_lqr = np.linalg.solve(R_lqr + B_d.T @ P_dare @ B_d, B_d.T @ P_dare @ A_d)

def lqr_control(x, x_ref, u_ref, K, u_max):
    e = state_error(x, x_ref)
    u = u_ref - K @ e
    return np.clip(u, -u_max, u_max)

# QP-MPC setup
Q_mpc = np.diag([10.0, 10.0, 5.0, 0.1, 0.1])
R_mpc = np.diag([0.1, 0.1])
R_rate = np.diag([2.0, 2.0])
N_horizon = 15

mpc_qp = QPMPC(L, tau, dt, Q_mpc, R_mpc, R_rate, N_horizon, u_max, exp_weight=1.0)

# NL-MPC setup
Q_nl = np.diag([10.0, 10.0, 5.0, 0.1, 0.1])
R_nl = np.diag([0.1, 0.1])
N_horizon_nl = 10
N_control_nl = 5

# Create dynamics function for NL-MPC
def dynamics_for_mpc(x, u, dt_step):
    return nonlinear_dynamics(x, u, dt_step, L, tau)

mpc_nl = NonlinearMPC(dynamics_for_mpc, dt, Q_nl, R_nl, N_horizon_nl, N_control_nl, u_max)

print("Controllers initialized")

## Reference Trajectory

In [ ]:
waypoints = np.array([
    [0.0, 0.0],
    [3.0, 3.0],
    [6.0, 2.0],
    [8.0, 5.0],
])

x_ref, u_ref = generate_smooth_reference_trajectory(waypoints, t, v_cruise, L, tau, turn_time=1.0)

plt.figure(figsize=(8, 6))
plt.plot(x_ref[0, :], x_ref[1, :], 'k--', linewidth=1.5, label='Reference')
plt.plot(waypoints[:, 0], waypoints[:, 1], 'ko', markersize=10)
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.legend()
plt.title('Reference Trajectory')
plt.show()

## Simulation Function

Run closed-loop simulation with EKF state estimation. The controller uses the EKF estimate (not the true state).

In [ ]:
def run_simulation(controller_name, controller_fn, x_ref, u_ref, waypoints,
                   camera_enabled=True, goal_tol=0.1, seed=42):
    """
    Run closed-loop simulation with EKF estimation.
    
    Parameters
    ----------
    controller_name : str
        Name for display
    controller_fn : callable
        Function (x_hat, x_ref, u_ref, k) -> u
    x_ref, u_ref : arrays
        Reference trajectory
    waypoints : array
        Waypoints for goal detection
    camera_enabled : bool
        Whether to use camera updates
    goal_tol : float
        Terminate when within this distance of final waypoint
    seed : int
        Random seed
        
    Returns
    -------
    dict with x_true, x_hat, P_hist, u_hist, t
    """
    rng = np.random.default_rng(seed)
    final_waypoint = waypoints[-1]
    
    # Initial state and covariance
    x0 = x_ref[:, 0].copy()
    P0 = np.diag([0.01, 0.01, 0.01, 0.001, 0.001])
    
    # Initialize EKF
    ekf = MultiRateEKF(L, tau, dt, Q_process, R_imu, R_camera, x0, P0)
    
    # Storage
    x_true = np.zeros((n_x, N))
    x_hat = np.zeros((n_x, N))
    P_hist = np.zeros((n_x, N))
    u_hist = np.zeros((n_u, N))
    
    # Initial conditions
    x_true[:, 0] = x0
    x_hat[:, 0] = ekf.get_state()
    P_hist[:, 0] = np.diag(ekf.get_covariance())
    
    # Measurement noise generators
    R_imu_chol = np.linalg.cholesky(R_imu)
    R_cam_chol = np.linalg.cholesky(R_camera)
    
    final_k = N - 1
    for k in range(N - 1):
        # Controller uses EKF estimate
        u_k = controller_fn(ekf.get_state(), x_ref[:, k], u_ref[:, k], k)
        u_hist[:, k] = u_k
        
        # True system propagation with process noise
        w = Q_chol @ rng.standard_normal(n_x)
        x_true[:, k+1] = nonlinear_dynamics(x_true[:, k], u_k, dt, L, tau) + w
        x_true[2, k+1] = angle_wrap(x_true[2, k+1])
        
        # Generate measurements from true state
        # IMU + encoder + accelerometer (always available)
        theta_true = x_true[2, k+1]
        v_l_true, v_r_true = x_true[3, k+1], x_true[4, k+1]
        theta_dot_true = (v_r_true - v_l_true) / L
        a_x_true = (1 / (2*tau)) * ((u_k[0] - v_l_true) + (u_k[1] - v_r_true))
        a_y_true = ((v_l_true + v_r_true) / 2) * theta_dot_true
        
        y_imu = np.array([theta_true, theta_dot_true, v_l_true, v_r_true, a_x_true, a_y_true])
        y_imu += R_imu_chol @ rng.standard_normal(6)
        
        # EKF predict
        ekf.predict(u_k)
        
        # EKF update with IMU/encoder/accel
        ekf.update_imu_encoder(y_imu, u_k)
        
        # Camera update (if enabled and at camera rate)
        if camera_enabled and (k+1) % camera_interval == 0:
            y_cam = x_true[:2, k+1] + R_cam_chol @ rng.standard_normal(2)
            ekf.update_camera(y_cam)
        
        # Store
        x_hat[:, k+1] = ekf.get_state()
        P_hist[:, k+1] = np.diag(ekf.get_covariance())
        
        # Early termination when goal reached
        dist_to_goal = np.sqrt((x_true[0, k+1] - final_waypoint[0])**2 + 
                               (x_true[1, k+1] - final_waypoint[1])**2)
        if dist_to_goal < goal_tol and k > 50:
            final_k = k + 1
            break
    
    # Trim to actual length
    x_true = x_true[:, :final_k + 1]
    x_hat = x_hat[:, :final_k + 1]
    P_hist = P_hist[:, :final_k + 1]
    u_hist = u_hist[:, :final_k + 1]
    t_sim = t[:final_k + 1]
    
    return {
        'x_true': x_true,
        'x_hat': x_hat,
        'P_hist': P_hist,
        'u_hist': u_hist,
        't': t_sim,
        'name': controller_name,
        'camera': camera_enabled
    }

## Run Simulations

Compare all three controllers with and without camera.

In [ ]:
# Define controller functions
def lqr_fn(x_hat, x_r, u_r, k):
    return lqr_control(x_hat, x_r, u_r, K_lqr, u_max)

def qpmpc_fn(x_hat, x_r, u_r, k):
    k_end = min(k + N_horizon + 1, N)
    x_ref_h = x_ref[:, k:k_end]
    u_ref_h = u_ref[:, k:k_end]
    if x_ref_h.shape[1] < N_horizon + 1:
        pad = N_horizon + 1 - x_ref_h.shape[1]
        x_ref_h = np.hstack([x_ref_h, np.tile(x_ref[:, -1:], (1, pad))])
        u_ref_h = np.hstack([u_ref_h, np.tile(u_ref[:, -1:], (1, pad))])
    return mpc_qp.compute_control(x_hat, x_ref_h, u_ref_h)

def nlmpc_fn(x_hat, x_r, u_r, k):
    k_end = min(k + N_horizon_nl + 1, N)
    x_ref_h = x_ref[:, k:k_end]
    u_ref_h = u_ref[:, k:k_end]
    if x_ref_h.shape[1] < N_horizon_nl + 1:
        pad = N_horizon_nl + 1 - x_ref_h.shape[1]
        x_ref_h = np.hstack([x_ref_h, np.tile(x_ref[:, -1:], (1, pad))])
        u_ref_h = np.hstack([u_ref_h, np.tile(u_ref[:, -1:], (1, pad))])
    return mpc_nl.compute_control(x_hat, x_ref_h, u_ref_h)

# Run all simulations
print("Running simulations...")
print("="*50)

results = {}

# LQR
print("LQR+FF without camera...")
results['lqr_nocam'] = run_simulation('LQR+FF', lqr_fn, x_ref, u_ref, waypoints, camera_enabled=False, seed=42)
print(f"  -> {len(results['lqr_nocam']['t'])} steps ({results['lqr_nocam']['t'][-1]:.1f}s)")
print("LQR+FF with camera...")
results['lqr_cam'] = run_simulation('LQR+FF', lqr_fn, x_ref, u_ref, waypoints, camera_enabled=True, seed=42)
print(f"  -> {len(results['lqr_cam']['t'])} steps ({results['lqr_cam']['t'][-1]:.1f}s)")

# QP-MPC
print("QP-MPC without camera...")
mpc_qp.reset()
results['qpmpc_nocam'] = run_simulation('QP-MPC', qpmpc_fn, x_ref, u_ref, waypoints, camera_enabled=False, seed=42)
print(f"  -> {len(results['qpmpc_nocam']['t'])} steps ({results['qpmpc_nocam']['t'][-1]:.1f}s)")
print("QP-MPC with camera...")
mpc_qp.reset()
results['qpmpc_cam'] = run_simulation('QP-MPC', qpmpc_fn, x_ref, u_ref, waypoints, camera_enabled=True, seed=42)
print(f"  -> {len(results['qpmpc_cam']['t'])} steps ({results['qpmpc_cam']['t'][-1]:.1f}s)")

# NL-MPC
print("NL-MPC without camera...")
mpc_nl.reset()
results['nlmpc_nocam'] = run_simulation('NL-MPC', nlmpc_fn, x_ref, u_ref, waypoints, camera_enabled=False, seed=42)
print(f"  -> {len(results['nlmpc_nocam']['t'])} steps ({results['nlmpc_nocam']['t'][-1]:.1f}s)")
print("NL-MPC with camera...")
mpc_nl.reset()
results['nlmpc_cam'] = run_simulation('NL-MPC', nlmpc_fn, x_ref, u_ref, waypoints, camera_enabled=True, seed=42)
print(f"  -> {len(results['nlmpc_cam']['t'])} steps ({results['nlmpc_cam']['t'][-1]:.1f}s)")

print("="*50)
print("Done!")

## Results

### Trajectory Comparison

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

controllers = [('lqr', 'LQR+FF'), ('qpmpc', 'QP-MPC'), ('nlmpc', 'NL-MPC')]

for ax, (key, name) in zip(axes, controllers):
    r_nocam = results[f'{key}_nocam']
    r_cam = results[f'{key}_cam']
    
    ax.plot(x_ref[0, :], x_ref[1, :], 'k--', linewidth=1, alpha=0.5, label='Reference')
    ax.plot(r_nocam['x_true'][0, :], r_nocam['x_true'][1, :], 'r-', linewidth=1.5, alpha=0.7, label='No camera')
    ax.plot(r_cam['x_true'][0, :], r_cam['x_true'][1, :], 'b-', linewidth=1.5, alpha=0.7, label='With camera')
    ax.plot(waypoints[:, 0], waypoints[:, 1], 'ko', markersize=8)
    
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.axis('equal')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper left')
    ax.set_title(name)

plt.tight_layout()
plt.show()

### Tracking Error

In [ ]:
def compute_tracking_error(result, x_ref):
    n = len(result['t'])
    return np.sqrt((result['x_true'][0, :n] - x_ref[0, :n])**2 + 
                   (result['x_true'][1, :n] - x_ref[1, :n])**2)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (key, name) in zip(axes, controllers):
    r_nocam = results[f'{key}_nocam']
    r_cam = results[f'{key}_cam']
    
    err_nocam = compute_tracking_error(r_nocam, x_ref)
    err_cam = compute_tracking_error(r_cam, x_ref)
    
    rms_nocam = np.sqrt(np.mean(err_nocam**2))
    rms_cam = np.sqrt(np.mean(err_cam**2))
    
    ax.plot(r_nocam['t'], err_nocam*100, 'r-', linewidth=1.5, alpha=0.7, 
            label=f'No camera ({rms_nocam*100:.1f} cm RMS)')
    ax.plot(r_cam['t'], err_cam*100, 'b-', linewidth=1.5, alpha=0.7, 
            label=f'With camera ({rms_cam*100:.1f} cm RMS)')
    
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Position Error [cm]')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_title(f'{name} Tracking Error')

plt.tight_layout()
plt.show()

### Position Uncertainty

The key benefit of camera updates: bounded position uncertainty. Without camera, uncertainty grows unbounded (dead reckoning drift).

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (key, name) in zip(axes, controllers):
    r_nocam = results[f'{key}_nocam']
    r_cam = results[f'{key}_cam']
    
    P_nocam = r_nocam['P_hist']
    P_cam = r_cam['P_hist']
    
    # Position uncertainty (x and y)
    sigma_xy_nocam = np.sqrt(P_nocam[0, :] + P_nocam[1, :])
    sigma_xy_cam = np.sqrt(P_cam[0, :] + P_cam[1, :])
    
    ax.plot(r_nocam['t'], sigma_xy_nocam*100, 'r-', linewidth=1.5, alpha=0.7, label='No camera')
    ax.plot(r_cam['t'], sigma_xy_cam*100, 'b-', linewidth=1.5, alpha=0.7, label='With camera')
    
    # Mark camera update times (on shorter array)
    t_cam = r_cam['t']
    cam_times = t_cam[::camera_interval]
    for ct in cam_times[1::5]:  # Every 5th for clarity
        ax.axvline(ct, color='gray', linestyle=':', alpha=0.3, linewidth=0.5)
    
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Position Uncertainty [cm]')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_title(f'{name} Position Uncertainty (1-$\\sigma$)')

plt.tight_layout()
plt.show()

### Estimation Error

How well does the EKF estimate match the true state?

In [ ]:
def compute_estimation_error(result):
    return np.sqrt((result['x_true'][0, :] - result['x_hat'][0, :])**2 + 
                   (result['x_true'][1, :] - result['x_hat'][1, :])**2)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (key, name) in zip(axes, controllers):
    r_nocam = results[f'{key}_nocam']
    r_cam = results[f'{key}_cam']
    
    est_err_nocam = compute_estimation_error(r_nocam)
    est_err_cam = compute_estimation_error(r_cam)
    
    ax.plot(r_nocam['t'], est_err_nocam*100, 'r-', linewidth=1.5, alpha=0.7, label='No camera')
    ax.plot(r_cam['t'], est_err_cam*100, 'b-', linewidth=1.5, alpha=0.7, label='With camera')
    
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Estimation Error [cm]')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_title(f'{name} Position Estimation Error')

plt.tight_layout()
plt.show()

## Summary

In [ ]:
print("\n" + "="*80)
print("MULTI-RATE SENSOR FUSION RESULTS")
print("="*80)
print(f"\n{'Controller':<12} {'Camera':<10} {'Track RMS':<12} {'Est RMS':<12} {'Final P_xy':<12}")
print("-"*80)

for key, name in controllers:
    for suffix, cam_str in [('_nocam', 'No'), ('_cam', 'Yes')]:
        r = results[f'{key}{suffix}']
        track_err = compute_tracking_error(r, x_ref)
        est_err = compute_estimation_error(r)
        final_P = np.sqrt(r['P_hist'][0, -1] + r['P_hist'][1, -1])
        
        print(f"{name:<12} {cam_str:<10} {np.sqrt(np.mean(track_err**2))*100:>6.2f} cm   "
              f"{np.sqrt(np.mean(est_err**2))*100:>6.2f} cm   {final_P*100:>6.2f} cm")

print("="*80)
print("\nKey observations:")
print("- Camera updates bound position uncertainty (dead reckoning drift eliminated)")
print("- Lower estimation error leads to better tracking (feedback uses estimate)")
print("- All controllers benefit from better state estimates")
print("- MPC controllers can anticipate and handle estimation uncertainty better")

## Conclusion

Multi-rate sensor fusion with the EKF provides:

1. **Bounded uncertainty** — Camera updates prevent unbounded drift from dead reckoning
2. **Better tracking** — Accurate state estimates enable effective feedback control
3. **Flexibility** — The EKF naturally handles sensors at different rates

The accelerometer provides redundant velocity information that helps between camera updates, but cannot prevent position drift on its own since it doesn't directly observe position.

For the Benji rover:
- IMU + encoders + accelerometer at 50 Hz for smooth control
- Camera at 5-10 Hz to bound position uncertainty
- All three controllers benefit from camera-aided navigation